In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_excel('/Users/aiakovleva/Downloads/comments.xlsx')
df.head

In [ ]:
name = []
reviews = []
grade = []


for element in range(63):
    na = df['Name'][element]
    review = df['Comment'][element]
    gr = df['Grade'][element]
    name.append(na)
    reviews.append(review)
    grade.append(gr)

df = pd.DataFrame({'Name': name, 'Comment': reviews, 'Grade': grade})
df 

In [ ]:
for i in range (len(reviews)):
    print(reviews[i])

In [ ]:
df1 = df.copy()
name1 = []
data = {}

for row in range(len(df1.index)):
    if df1.iat[row, 0] not in name1:
        m=name1.append(df1.iat[row, 0])
        df2 = df1[df1['Name'] == df1.iat[row, 0]]
        tem = []
        for i in range(len(df2.index)):
            tem.append(df2.iat[i, 1].lower())
        data[df1.iat[row, 0]] = tem

reviews1 = []*len(name1)

for i in range(len(name1)):
    reviews1.append(data[name1[i]])

In [ ]:
#for i in range(len(name1)):
#    name1[i] = name1[i][:-1]

In [ ]:
import re
import string

In [ ]:
import nltk
nltk.download('punkt')
import pymorphy2
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocessing(review):
    
    # data cleaning
    review = review.lower()
    #review = re.sub(r'((www.\.\[^\s]+)|(https?://[^\s]+))', r'', review)
    #review = re.sub(r'@[^\s]+', r'', review)
    #review = re.sub(r'\W*\b\w{1,3}\b', r'', review)
    
    token = nltk.word_tokenize(review)
    
    #review = [word for word in token if (word not in string.punctuation and word != "' '" and word != "``" and word != "''" and word.isnumeric() == False) ]
    
    review = ' '.join(pymorphy2.MorphAnalyzer().parse(word)[0].normal_form for word in review)
#     review = [lemmatizer.lemmatize(word) for word in review]
#     review = ' '.join(review)
    
    return review 

In [ ]:
def prepareOnePlaceReviews(reviews):
    for i in range(len(reviews)):
        reviews[i] = preprocessing(reviews[i])
    return reviews

In [ ]:
#df['Comment'] = df['Comment'].apply(lambda reviews: prepareOnePlaceReviews(reviews))

In [ ]:
new_temp = df.copy()

In [ ]:
df['Comment'][0]

In [ ]:
import numpy as np
import re
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
f = open('/Users/aiakovleva/Downloads/train_data_true.txt', 'r', encoding='utf-8')
texts_true = f.readlines()
texts_true[0] = texts_true[0].replace('\ufeff','')

In [ ]:
f = open('/Users/aiakovleva/Downloads/train_data_false.txt', 'r', encoding='utf-8')
texts_false = f.readlines()
texts_false[0] = texts_false[0].replace('\ufeff','')

In [ ]:
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true,count_false,total_lines)

In [ ]:
maxWordsCount = 2000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer. fit_on_texts (texts)

In [ ]:
data = tokenizer.texts_to_sequences(texts)
max_text_len = 10
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)
print(data_pad.shape)

In [ ]:
X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

In [ ]:
indeces = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indeces]
Y = Y[indeces]

In [ ]:
model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length = max_text_len))
model.add(LSTM(64, activation='tanh', return_sequences=True))
model.add(LSTM(32, activation='tanh'))
model.add(Dense(2, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer = Adam(0.0001))

In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=50)

In [ ]:
strings = df.shape[0]
print(strings)

In [ ]:
def res_to_string(result):
    count_all_positive = 0
    count_all_negative = 0
    if (result[0][0] < result[0][1]):
        count_all_negative += 1;
    elif (result[0][0]) > (result[0][1]):
        count_all_positive += 1;
    return [count_all_positive,count_all_negative]

In [ ]:
tup_pos = []
tup_neg = []
tup = []

for i in range(strings):
    a = 0
    b = 0
    tup_tonality = []*len(df.iat[i,1])
    for j in range(len(df.iat[i,1])):
        t = df.iat[i,1][j]
        data = tokenizer.texts_to_sequences([t])
        data_pad = pad_sequences(data, maxlen = max_text_len)
        res = model.predict(data_pad)
        tup_tonality.append(res_to_string(res))
        print(res_to_string(res))
        a += res_to_string(res)[0]
        b += res_to_string(res)[1]
    tup_pos.append(a)
    tup_neg.append(b)
    tup.append(tup_tonality)

In [ ]:
#new_temp = df[['Name', 'Comment']]
#new_temp['tonality_of_each_review'] = tup
#new_temp

In [ ]:
positive = []
negative = []

for p in range(len(new_temp.index)):
    
    pos_reviews = []
    neg_reviews = []

    for k in range(len(df.iat[p,1])):
        if df.iat[p,1][k] == [1,0]:
            pos_reviews.append(df.iat[p,1][k])
        else:
            neg_reviews.append(df.iat[p,1][k])
    positive.append(pos_reviews)
    negative.append(neg_reviews)


new_temp['positive_reviews'] = positive
new_temp['negative_reviews'] = negative
# new_temp

In [ ]:
ready_for_clouds = new_temp[['Name', 'positive_reviews', 'negative_reviews']]
ready_for_clouds

In [ ]:
df['Number of positive reviews'] = tup_pos
df['Number of negative reviews'] = tup_neg
for i in range(strings):
    if new.iat[i,1] == ['']:
        new.iat[i,3] = new.iat[i,2]

df['Total number of reviews'] = df['Number of positive reviews'] + df['Number of negative reviews']
df

In [ ]:
conditions = [(df['Number of positive reviews'] * 5 + df['Number of negative reviews'] * 1)/(df['Number of negative reviews']+df['Number of positive reviews'])<=1.5,
              (df['Number of positive reviews'] * 5 + df['Number of negative reviews'] * 1)/(df['Number of negative reviews']+df['Number of positive reviews'])<=2.5, 
              (df['Number of positive reviews'] * 5 + df['Number of negative reviews'] * 1)/(df['Number of negative reviews']+df['Number of positive reviews'])<=3.5, 
              (df['Number of positive reviews'] * 5 + df['Number of negative reviews'] * 1)/(df['Number of negative reviews']+df['Number of positive reviews'])<=4.5 ,
             (df['Number of positive reviews'] * 5 + df['Number of negative reviews'] * 1)/(df['Number of negative reviews']+df['Number of positive reviews'])>=4.5]
five, four, three, two, one = 5, 4, 3, 2, 1
output = [one, two, three, four, five]
df['Rating'] = np.select(conditions, output)

In [ ]:
df['Final Grade'] = []*strings
for i in range(strings):
    if df['Grade'][i] != '/':
        df['Final Grade'][i] = (df['Grade'][i]+df['Rating'][i])/2
    else:
        df['Final Grade'][i] = df['Rating'][i]
        
df